In [23]:
import pandas as pd
import numpy as np


## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
ratings = pd.read_csv('ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [40]:
# Считаем нужные метрики - дата первой и последней оценки, а также количество выставленных оценок
ratings_count = ratings.groupby(['userId']).agg({'rating': 'count', 'timestamp': ['min', 'max']}).reset_index()[ratings_count['rating']['count'] > 100]

In [56]:
# Добавляем к датафрейму столбец с разницей даты последней и первой оценки, а также считаем среднее значение жизни пользователей на сайте
ratings_count['lifetime'] =  ratings_count['timestamp']['max'] - ratings_count['timestamp']['min']
mean = ratings_count['lifetime'].mean()
ratings_count.head()

userId rating   timestamp               lifetime
          count         min         max           
1      2    184  1141415509  1141418008       2499
2      3    656  1439472199  1566091883  126619684
3      4    242  1573937091  1574195101     258010
4      5    101   830786155   858628421   27842266
7      8    155   890489203   891279691     790488

In [60]:
print(f'Среднее время жизни пользователей, которые проставили больше 100 оценок составляет {round(mean/3600,2)} часов, или {int(mean/86400)} дней')

Среднее время жизни пользователей, которые проставили больше 100 оценок составляет 11860.43 часов, или 494 дней


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [61]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [62]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [63]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [64]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [69]:
# Таблица без адреса
types_of_rvenue = client_base.merge(rzd, how='left', on='client_id').merge(auto, how='left', on='client_id').merge(air, how='left', on='client_id').drop('address', axis = 'columns')
types_of_rvenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [70]:
# Таблица с адресом
types_of_rvenue = client_base.merge(rzd, how='left', on='client_id').merge(auto, how='left', on='client_id').merge(air, how='left', on='client_id')
types_of_rvenue

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Для того, чтобы добавить координаты к таблице визитов желательно, чтобы и в таблице координат и в таблице визитов присутствовала колонка времени, или же id визита, тогда можно будет точно определить координаты во время визита сайта.
Такая же логика будет по таблице покупок, здесь вохможно также получить время из таблицы транзакций, если такой имеет место быть.

Если расчитывать в логике, определенной выше, то такой проблемы не должно существовать.
Если предположить, что у нас нет id визитов, нет времени визита и времени предоставления координат, то задача усложняется, тк у нас есть возможность добавить координаты только по id клиента.

По id координат мы сможем определить в каких регионах повышенный спрос на конкретные запросы/покупки, в связи с чем сможем настроить верно рекламу, определить запасы тех или иных товаров, вернее выстраивать логистику и строить дарксторы или склады (если это наше направление), все это в свою очередь имеет потенциал к снижению операционных расходов, увеличению продаж, верному распределению капитальных затрат, что влияет на увеличение прибыли.